# Sentinel-2 Sunglint Finder with Earth Engine Python API

This notebook shows how to find Sentinel-2 scenes over sunglint geometry using the Earth Engine Python API.  
Adjust the `bbox`, `start_date`, `end_date`, and other parameters as needed.


In [ ]:
# 1. Install and import Earth Engine Python API
# Uncomment the following line if running in Colab or a new environment:
# !pip install earthengine-api

import ee
import math

# Initialize the Earth Engine client. 
# If running for the first time, you may need to authenticate:
# ee.Authenticate()
ee.Initialize()
print('Earth Engine initialized.')


In [ ]:
# West Africa Tropical Atlantic (off Senegal to Angola)
aoi_west_africa = ee.Geometry.Rectangle([-30.0, 0.0, -10.0, 15.0])

# Equatorial Atlantic (Gulf of Guinea)
aoi_equatorial_atlantic = ee.Geometry.Rectangle([-10.0, -5.0, 10.0, 10.0])

# Eastern Pacific (off Central America)
aoi_eastern_pacific = ee.Geometry.Rectangle([-110.0, -5.0, -90.0, 10.0])

# Western Indian Ocean (near Madagascar and Mozambique Channel)
aoi_indian_ocean = ee.Geometry.Rectangle([35.0, -20.0, 60.0, 0.0])

# Equatorial Pacific (near Kiribati, remote tropical waters)
aoi_central_pacific = ee.Geometry.Rectangle([-180.0, -10.0, -160.0, 10.0])

# Bay of Bengal (Eastern Indian Ocean, often glinty)
aoi_bay_of_bengal = ee.Geometry.Rectangle([80.0, 5.0, 95.0, 20.0])

# South China Sea (tropical glint-prone waters)
aoi_south_china_sea = ee.Geometry.Rectangle([110.0, 0.0, 125.0, 15.0])


brend_example = -91.469444, 28.430278
brend_example_date = "5-15-2022"

In [ ]:
coords = [
    [102.98827, 7.59247],
    [102.98800, 7.59238],
    [102.98827, 7.59283],
    [102.98799, 7.59346],
    [102.98827, 7.59265]
]

# Create a MultiPoint geometry
stationary_infra = ee.Geometry.MultiPoint(coords)

In [ ]:


# Define a wide sunglint-prone AOI (off West Africa, open tropical ocean)
# aoi = ee.Geometry.Rectangle([-30.0, 0.0, -10.0, 15.0])

# 1. Filter Sentinel-2 SR Harmonized collection
s2 = ee.ImageCollection('COPERNICUS/S2') \
    .filterDate('2020-01-01', '2024-12-01') \
    .filterBounds(stationary_infra) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
    # .filter(ee.Filter.gt('MEAN_SOLAR_ZENITH_ANGLE', 60))  # Stronger glint

In [ ]:
# def mask_s2_clouds(img):
#     # Use MSK_CLASSI_OPAQUE (cloud) and MSK_CLASSI_CIRRUS (cirrus) bands
#     opaque = img.select('MSK_CLASSI_OPAQUE')
#     cirrus = img.select('MSK_CLASSI_CIRRUS')
#     mask = opaque.eq(0).And(cirrus.eq(0))
#     return img.updateMask(mask)

# s2 = s2.map(mask_s2_clouds)

In [ ]:


# 2. Add optional Sun Glint Index (green vs SWIR)
def add_sgi(img):
    green = img.select('B3').divide(10000)
    swir = img.select('B11').divide(10000)
    sgi = green.subtract(swir).divide(green.add(swir)).rename('SGI')
    return img.addBands(sgi)

s2 = s2.map(add_sgi)

# 3. Estimate sunglint pixel fraction from SWIR reflectance (BOEM method)
def add_glint_fraction(img):
    swir = img.select('B11').divide(10000)
    glint_mask = swir.gt(0.13)  # SWIR glint threshold
    glint_fraction = glint_mask.reduceRegion(
        reducer=ee.Reducer.mean(),
        scale=100,
        maxPixels=1e14
    ).get('B11')
    return img.set('glint_fraction', glint_fraction)

s2 = s2.map(add_glint_fraction)


def add_sgi_median(img):
    green = img.select('B3').divide(10000)
    swir = img.select('B11').divide(10000)
    
    # Avoid divide-by-zero errors
    sgi = green.subtract(swir).divide(green.add(swir)).rename('SGI')
    
    # Mask invalid values
    valid_sgi = sgi.updateMask(sgi.gte(-1).And(sgi.lte(1)))

    sgi_median = valid_sgi.reduceRegion(
        reducer=ee.Reducer.median(),
        # geometry=aoi,
        scale=20,
        maxPixels=1e24
    ).get('SGI')

    return img.set('SGI_median', sgi_median)

# Map the function over your filtered S2 collection
s2_with_sgi = s2.map(add_sgi_median)

# Now filter by SGI_mean (a real image property)
glint_images = s2_with_sgi.filter(ee.Filter.gt('SGI_max', 0.7))  # lower SGI = stronger glint


# 4. Filter for strong sunglint images (e.g., >20% of pixels glinted)
# glint_images = s2.filter(ee.Filter.gt('SGI', 0.0))

# # 5. Get top image and print properties
# top_image = glint_images.sort('glint_fraction', False).first()
# info = top_image.select(['B4', 'B3', 'B2', 'SGI']).getInfo()

# # 6. Export info or use in visualization tool (e.g., geemap, folium, etc.)
# print("Top sunglint image ID:", top_image.get('system:index').getInfo())
# print("Sunglint fraction:", top_image.get('glint_fraction').getInfo())


In [ ]:
s2.size()

In [ ]:
high_glint_fract = s2.sort('glint_fraction', False).first()
high_sgi = s2.sort('SGI_median', False).first()

In [ ]:
# # Mask glint_images to ocean only using a simple landmask (e.g., GSHHS or MODIS water mask)
# # Here we use the JRC Global Surface Water mask (water = 1, land = 0)
# water_mask = ee.Image('JRC/GSW1_4/GlobalSurfaceWater').select('occurrence').gt(0)

# def mask_land(img):
#     return img.updateMask(water_mask)

# glint_images_ocean = glint_images.map(mask_land)

In [ ]:
import geemap

Map = geemap.Map(center=[7.5, -20], zoom=5)
# Map.addLayer(s2.median().select('SGI'), {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'red']}, 'SGI')
# Map.addLayer(s2.median().visualize(bands=['B4', 'B3', 'B2'], min=0, max=3000), {}, 'RGB')
# Map
# Add AOIs as layers to the map for reference
# Map.addLayer(aoi_west_africa, {'color': 'yellow'}, 'AOI West Africa')
# Map.addLayer(aoi_equatorial_atlantic, {'color': 'orange'}, 'AOI Equatorial Atlantic')
# Map.addLayer(aoi_eastern_pacific, {'color': 'cyan'}, 'AOI Eastern Pacific')
# Map.addLayer(aoi_indian_ocean, {'color': 'magenta'}, 'AOI Indian Ocean')
# Map.addLayer(aoi_central_pacific, {'color': 'lime'}, 'AOI Central Pacific')
# Map.addLayer(aoi_bay_of_bengal, {'color': 'red'}, 'AOI Bay of Bengal')
# Map.addLayer(aoi_south_china_sea, {'color': 'blue'}, 'AOI South China Sea')


Map.addLayer(high_glint_fract.visualize(bands=['B4', 'B3', 'B2'], min=0, max=3000), {}, 'glint')
Map.addLayer(high_sgi.visualize(bands=['B4', 'B3', 'B2'], min=0, max=3000), {}, 'high SGI glint')
Map.addLayer(high_glint_fract.select('SGI'), {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'red']}, 'glint SGI')


Map.addLayer(stationary_infra,{'color': 'red'}, 'Stationary')
Map.centerObject(stationary_infra)
Map

In [ ]:
# glint_images_ocean.size()